# **Augmentation and Analysis Tools for Intellegent Origami Detection**


---


Here we give use examples of an CLI augmentation tool we developed to prepare out data for YOLOv5 training. 

We also give an example of CLI annotation analysis tool we developed to quickly process some of our results. 

# Clone in Repo, Setup, and Dependancies


---



In [1]:
# Mount google drive for saving the results

from google.colab import drive
drive.mount('/content/gdrive')

print('Done.')

Mounted at /content/gdrive
Done.


In [1]:
### Clone in repo
!git clone https://github.com/mchirib1/Origami_Structure_Detection.git
%cd /content

!pip install -r Origami_Structure_Detection/requirements.txt  # install dependencies
print('Setup complete.') # A restart runtime message might appear... To continue, select restart runtime and re-run this code block

fatal: destination path 'Origami_Structure_Detection' already exists and is not an empty directory.
/content
Setup complete.


## Example Source Files


---


Example source files are included in the github repo.  However it is trivial to assign a new source. Simply make a new directory and pass it's path to the `--src` argument

In [8]:
# list source files in src directory
import os

os.chdir('/content/Origami_Structure_Detection/src') # src is default source directory
for file in sorted(os.listdir()):
  print(file) # There should be 10 images, each with an associated text file

print('-------------------------------------------------')
print(f'A total of {len(os.listdir())} files found.') # Should be 20 files total

triangle_1.png
triangle_1.txt
triangle_10.png
triangle_10.txt
triangle_2.png
triangle_2.txt
triangle_3.png
triangle_3.txt
triangle_4.png
triangle_4.txt
triangle_5.png
triangle_5.txt
triangle_6.png
triangle_6.txt
triangle_7.png
triangle_7.txt
triangle_8.png
triangle_8.txt
triangle_9.png
triangle_9.txt
-------------------------------------------------
A total of 20 files found.


# Augmentation Tool


---


First we look at the augmentation tool.  It has a total of 7 optional arguments each with a description which can be seen below. 

In this notebook we include examples of the first 5. 

In [9]:
%cd /content/Origami_Structure_Detection

# list the arguments for the augment.py script
!python augment.py --help

/content/Origami_Structure_Detection
usage: augment.py [-h] [--src source_path] [--dst dest_path] [--cvt_gray]
                  [--cvt_ab alpha beta] [--rot n theta] [--quad1] [--quad3]

CLI tool for augmenting source images and corresponding annotations beore
feeding into the YOLO machine learning framework.

optional arguments:
  -h, --help           show this help message and exit
  --src source_path    Path to source directory containing image and label
                       stack
  --dst dest_path      Path to create a destination directory storing output
  --cvt_gray           Converts image stack to grayscale
  --cvt_ab alpha beta  Edits constants and brightness by scaling alpha and
                       beta
  --rot n theta        Rotates each image and corresponding label by random
                       angle between 1 and theta, n times
  --quad1              Sets all pixels outside of the upper left quadrant to
                       zero
  --quad3              Sets all 

## Conversion to Grayscale


---


The `--cvt_gray` argument converts images from a 3 dimensional $(x, y, 3)$ pixel array to a 2 dimensional $(x, y)$ pixel array. The $x$ and $y$ indeces refer to each pixels cartesian coordinates while the third index refers to the RGB color channels. If each channel has 8 bits, for red, green, and blue, each image can store discrete pixels with conventional brightness intensities between 0 and 255 for each of the color channels.

Conversion to  effectively eliminates the color channels associated with RGB images.



In [10]:
### Designate source and destination directories
src = 'src'
dst = 'grayscale'

%cd /content/Origami_Structure_Detection

!python augment.py --src $src --dst $dst --cvt_gray 

/content/Origami_Structure_Detection
Namespace(cvt_ab=False, cvt_gray=True, dst='grayscale', quad1=False, quad3=False, rot=False, src='src')
10 images converted to grayscale.
Code completed in 2.4442 seconds on 20210127.


## Alpha and Beta Conversion


---


The `--cvt_ab` argument modifies the each pixel's intensity according to the equation $g(x) = \alpha*f(x)+\beta$ where $g(x)$ is the modified pixel intensity and $f(x)$ is the source pixel intensity. 

Conventionally, modifying the contrast refers to scaling the $\alpha$ constant while the brightness refers to the $\beta$ constant.


---


**Warning:** *The nomenclature for the result files needs work. To avoid overwriting data, the modified file names include the assigned $\alpha$ and $\beta$ constants. If a non-integer value for $\alpha$ is assigned, the output file name automatically switches to "locon". This is in order to avoid using decimals in the file names.* 

*However, beware! This increases the possibility of overwitting data!!! If two different non-integer values of $\alpha$ are used with the the same $\beta$, one will be overwritten!!!*

In [15]:
### Designate source and destination 
src = 'src'
dst = 'contrast'

%cd /content/Origami_Structure_Detection

## Alpha and beta nomenclature scheme includes the factors in the resulting file
## names.  However it needs work and alpha values <1 are stored as "locon" to 
## avoid decimal file names. Though as long as a unique beta value is used, it 
## can be run multiple times. 

alpha = 3 
beta = 10
!python augment.py --src $src --dst $dst --cvt_ab $alpha $beta
print('-----------------------------------------------------------')
alpha = 0.75
beta = 50
!python augment.py --src $src --dst $dst --cvt_ab $alpha $beta
print('-----------------------------------------------------------')

/content/Origami_Structure_Detection
Namespace(cvt_ab=['3', '10'], cvt_gray=False, dst='contrast', quad1=False, quad3=False, rot=False, src='src')
10 images scaled using alpha=3.0 and beta=10.
Code completed in 3.9985 seconds on 20210127.
-----------------------------------------------------------
Namespace(cvt_ab=['0.75', '50'], cvt_gray=False, dst='contrast', quad1=False, quad3=False, rot=False, src='src')
10 images scaled using alpha=0.75 and beta=50.
Code completed in 3.9395 seconds on 20210127.
-----------------------------------------------------------


## Image Rotations


---


The `--rot` argument rotates the images in a directory along with accompanying annotations. The image is rotated by an angle randomly selected between 1 and a user selected $\theta$. In order to further augment the data set, the image can be rotated $n$ times.  

Thus, the default example is a source of 10 images. Each image is rotated $n=5$ times each time by a random angle $\theta$, such that $1 < \theta < 360$. This results in the generation of 50 unique images. 


In [12]:
### This step may be time consuming depending on the number of images ###
### Designate source and destination 
%cd /content/Origami_Structure_Detection

n_rotations = 5
theta = 360
dst = 'rotations' # all images will be stored in the same directory

src = 'src' # rotates unprocessed images
!python augment.py --src $src --dst $dst --rot $n_rotations $theta
print('-----------------------------------------------------------')
src = 'contrast' # rotates the images where we modified the contrast and brightness
!python augment.py --src $src --dst $dst --rot $n_rotations $theta
print('-----------------------------------------------------------')
src = 'grayscale' # rotates the images we converted to grayscale
!python augment.py --src $src --dst $dst --rot $n_rotations $theta
print('-----------------------------------------------------------')

/content/Origami_Structure_Detection
Namespace(cvt_ab=False, cvt_gray=False, dst='rotations', quad1=False, quad3=False, rot=['5', '360'], src='src')
10 images rotated 5 times. Output:50 images
Code completed in 28.6785 seconds on 20210127.
Namespace(cvt_ab=False, cvt_gray=False, dst='rotations', quad1=False, quad3=False, rot=['5', '360'], src='contrast')
20 images rotated 5 times. Output:100 images
Code completed in 58.3723 seconds on 20210127.
Namespace(cvt_ab=False, cvt_gray=False, dst='rotations', quad1=False, quad3=False, rot=['5', '360'], src='grayscale')
10 images rotated 5 times. Output:50 images
Code completed in 23.9473 seconds on 20210127.


# Analysis Tool


---


First we look at the analysis tool.  It has a total of 6 optional arguments each with a description which can be seen below. 

In [ ]:
%cd /content/Origami_Structure_Detection


# list the arguments for the augment.py script
!python analyze.py --help

## Counting Annotations and Distributions


---


The analysis tool offers a few different advantages:  

*   `--count_anns` and `--count_preds` can be used to count the annotations in each image. (Only difference is in nomenclature to avoid confusion)  
*   `--count_dims` can be used to report the shape of each image's pixel array
*   `--scat_hist` can be used to create a scatterplot with histograms on the axes to better visualize bounding box size and location distributions. 

**Notes:** *The `--count_anns` and `--count_preds` arguments return annotation counts for each individual image as well as sum for entire image stack. Also * 


In [ ]:
### Designate source and destination 
src = 'rotations'
dst = 'results'

%cd /content/Origami_Structure_Detection

!python analyze.py --src $src --dst $dst --count_anns --count_preds --count_dims --scat_hist

# Save Results to Google Drive


---


*Only available if the google drive mount key was used in the first code segment* 

This code segement saves the copies the current Origami_Structure_Detection directory into google drive. 

**Caution:** *Data may be overwritten!!*

In [ ]:
# predictions are found under the output folder and weights are stored in the pytorch file (".pt")
%cp  -r /content/Origami_Structure_Detection/ /content/gdrive/My\ Drive 

print("Done.")

Copied to google drive


# Optional...

This code block allows the user to clean up generated destination folders folders.  

**Warning:** *This will delete any unsaved data contained within!!!* 

In [32]:
### OPTIONAL ### empty or delete directories
%cd /content/Origami_Structure_Detection/

import shutil

dir_path = input('\nEnter path to directory')

try:
  shutil.rmtree(dir_path)
  print('\nData deleted.')
except FileNotFoundError:
  print('\nError: That directory does not exist.')
except NotADirectoryError:
  print('\nError: Path does not lead to directory.')



/content/Origami_Structure_Detection

Enter path to directory/content/Origami_Structure_Detection/results

Data deleted.
